In [2]:
import aif360
import holisticai
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from holisticai.bias.mitigation import Reweighing
from holisticai.bias.mitigation import LearningFairRepresentation

In [7]:
# sklearn imports
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# holisticai imports
from holisticai.bias.metrics import classification_bias_metrics

In [3]:
baseline_preprocessed_df = pd.read_csv("C:/Users/athar/OneDrive/Desktop/Rutgers/Ethical Stat Learning/Project/data/data_preprocessed_baseline.csv",index_col=0)
baseline_preprocessed_df = baseline_preprocessed_df.drop(['id','age_cat'],axis=1)

In [4]:
protected_variables = ["sex", "African-American_race"]
output_variable = ["two_year_recid"]

In [16]:
# Simple preprocessing
y = baseline_preprocessed_df[output_variable]
X = pd.get_dummies(baseline_preprocessed_df.drop(output_variable, axis=1))
group = ["African-American_race"]
group_a = baseline_preprocessed_df["African-American_race"] == 1
group_b = baseline_preprocessed_df["African-American_race"] == 0
data_ = [X, y, group_a, group_b]


In [17]:
# efficacy metrics from sklearn
from sklearn import metrics

# dictionnary of metrics
metrics_dict={
        "Accuracy": metrics.accuracy_score,
        "Balanced accuracy": metrics.balanced_accuracy_score,
        "Precision": metrics.precision_score,
        "Recall": metrics.recall_score,
        "F1-Score": metrics.f1_score}

# efficacy metrics dataframe helper tool
def metrics_dataframe(y_pred, y_true, metrics_dict=metrics_dict):
    metric_list = [[pf, fn(y_true, y_pred)] for pf, fn in metrics_dict.items()]
    return pd.DataFrame(metric_list, columns=["Metric", "Value"]).set_index("Metric")

In [18]:
# Train test split
dataset = train_test_split(*data_, test_size=0.2, shuffle=True,random_state=100)
train_data = dataset[::2]
test_data = dataset[1::2]

Baseline

In [19]:
# data and model
X_train, y_train, group_a_train, group_b_train = train_data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
model_lr = LogisticRegression()
model_lr.fit(X_train_scaled,y_train)

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [20]:
# predict test
X_test, y_test, group_a_test, group_b_test = test_data
X_test_scaled = scaler.transform(X_test)
y_pred = model_lr.predict(X_test_scaled)
y_proba = model_lr.predict_proba(X_test_scaled)
y_score = y_proba[:,1]
y_true = y_test

In [21]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.703549
Balanced accuracy,0.694090
Precision,0.702359
Recall,0.596302
F1-Score,0.645000


In [22]:
# Baseline Bias
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.236481,0
Disparate Impact,1.904347,1
Four Fifths Rule,0.525114,1
Cohen D,0.501399,0
2SD Rule,9.213965,0
Equality of Opportunity Difference,0.257901,0
False Positive Rate Difference,0.143088,0
Average Odds Difference,0.200494,0
Accuracy Difference,0.013012,0


Preprocessing

In [23]:
# initialise
rew = Reweighing()

In [24]:
# fit reweighing object to training data
X_train, y_train, group_a, group_b = train_data
rew.fit(y_train, group_a, group_b)

In [36]:
# access the new sample_weight
sw = rew.estimator_params["sample_weight"]
sw

array([1.1377071 , 1.1838248 , 0.88731617, ..., 0.8710808 , 0.88731617,
       0.8710808 ], dtype=float32)

In [26]:
# Implement a Logistic Regression (with Reweighing)

# train (with reweighing)
X, y, group_a, group_b = train_data
scaler = StandardScaler()
Xt = scaler.fit_transform(X)
model = LogisticRegression()
model.fit(Xt, y, sample_weight=sw) 

# test
X, y, group_a, group_b = test_data
Xt = scaler.transform(X)
y_pred = model.predict(Xt)
y_proba = model.predict_proba(Xt)
y_score = y_proba[:,1]
y_true = y

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [39]:
y_pred.shape

(1437,)

In [27]:
# efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.704245
Balanced accuracy,0.695268
Precision,0.700717
Recall,0.602465
F1-Score,0.647887


In [28]:
# bias
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.064818,0
Disparate Impact,1.182645,1
Four Fifths Rule,0.845562,1
Cohen D,0.133291,0
2SD Rule,2.519570,0
Equality of Opportunity Difference,0.066217,0
False Positive Rate Difference,-0.017118,0
Average Odds Difference,0.024549,0
Accuracy Difference,0.017148,0


LFR

In [46]:
# initialise
lfr = LearningFairRepresentation(k=5)

In [47]:
# fit lfr object to training data (remember to standard scale train data)
X_train, y_train, group_a, group_b = train_data
scaler1 = StandardScaler()
X_train_t = scaler1.fit_transform(X_train)
lfr.fit(X_train_t, y_train, group_a, group_b)

Optimization Progress:   0%|          | 0/5000 [00:00<?, ?it/s]

In [48]:
# transform train
X_train, y_train, group_a_train, group_b_train = train_data
X_train_t = scaler1.fit_transform(X_train)
new_X_train = lfr.transform(X_train_t, group_a_train, group_b_train)

# transform test
X_test, y_test, group_a_test, group_b_test = test_data
X_test_t = scaler1.fit_transform(X_test)
new_X_test = lfr.transform(X_test_t, group_a_test, group_b_test)

In [49]:
# Fit a model with new data (transformed by LFR algorithm)

# train
X, y, group_a, group_b = train_data
X = new_X_train
scaler2 = StandardScaler()
Xt = scaler2.fit_transform(X)
model = LogisticRegression()
model.fit(Xt, y)

# test
X, y, group_a, group_b = test_data
X = new_X_test
Xt = scaler2.transform(X)
y_pred = model.predict(Xt)
y_proba = model.predict_proba(Xt)
y_score = y_proba[:,1]
y_true = y

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
# efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.642310
Balanced accuracy,0.628876
Precision,0.634731
Recall,0.489985
F1-Score,0.553043


In [51]:
# bias
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.006532,0
Disparate Impact,0.981443,1
Four Fifths Rule,0.981443,1
Cohen D,-0.013708,0
2SD Rule,-0.259692,0
Equality of Opportunity Difference,0.036455,0
False Positive Rate Difference,-0.095774,0
Average Odds Difference,-0.029660,0
Accuracy Difference,0.036356,0
